In [ ]:
# Import relevant packages and fix the seeds for repeatability

import tensorflow as tf
import numpy as np
import pandas as pd
import random
import pickle
import keras
import os, time
from keras import backend as K
from keras.utils import np_utils
from IPython.display import clear_output
import matplotlib.pyplot as plt
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(int(time.time()))
random.seed(1)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1, gpu_options = gpu_options)
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
# Open the pre-processed data file and format it to satisfy the neural network

with open('train.pickle', 'rb') as f:
    X_train, Y_train = pickle.load(f)
    
X_train = X_train.reshape([X_train.shape[0], 4, 4, 1])

In [ ]:
# Filter out outliers using inter-quartile range
IQR = - np.quantile(Y_train, 0.25) + np.quantile(Y_train, 0.75)

lower_bound, upper_bound = np.quantile(Y_train, 0.25) - 1.5 * IQR, np.quantile(Y_train, 0.75) + 1.5 * IQR

idx, val = np.where((Y_train >= lower_bound) & (Y_train <= upper_bound))

Y_train = Y_train[idx]
X_train = X_train[idx]

In [ ]:
# Sanity check on the data shape and range of bandgaps
X_train.shape, Y_train.shape

In [ ]:
np.min(Y_train), np.max(Y_train)

* Here are one sample of the data, X_train[0] is a material structure while Y_train[0] is its corresponding bandgap

In [6]:
X_train[0]

array([[[-1.],
        [-1.],
        [-1.],
        [-1.]],

       [[-1.],
        [-1.],
        [-1.],
        [-1.]],

       [[-1.],
        [-1.],
        [-1.],
        [-1.]],

       [[-1.],
        [ 1.],
        [ 1.],
        [ 1.]]])

In [7]:
Y_train[0]

array([0.7233])

In [ ]:
# Create placeholders
real_data = tf.placeholder(tf.float32, shape = [None, 4, 4, 1])
z = tf.placeholder(tf.float32, shape = [None, 128])
y = tf.placeholder(tf.float32, [None, 1])

In [ ]:
# Leaky Relu
def lrelu(X, leak = 0.2):
    f1 = 0.5 * (1 + leak)
    f2 = 0.5 * (1 - leak)
    return f1 * X + f2 * tf.abs(X)

In [ ]:
# Structure of the generator
def generator(z, y_, reuse=None):
    with tf.variable_scope('G',reuse=reuse):      
        
        y_ = tf.concat([z, y_], axis = 1)
        
        hidden2 = tf.layers.dense(y_, units = 1 * 1 * 512)
        R1 = tf.reshape(hidden2, shape = [-1,1,1,512])
        
        for i in range(5):
            C1 = tf.layers.conv2d_transpose(R1, filters = 512, kernel_size = 3,
                                            strides = (1, 1), padding = 'same')
            B1 = tf.layers.batch_normalization(C1)
            R1 = lrelu(B1)
        
        C1 = tf.layers.conv2d_transpose(R1, filters = 256, kernel_size = 3,
                                        strides = (2, 2), padding = 'same')
        B1 = tf.layers.batch_normalization(C1)
        R1 = lrelu(B1)
        
        C2 = tf.layers.conv2d_transpose(R1, filters = 128, kernel_size = 3,
                                       strides = (2, 2), padding = 'same')
        B2 = tf.layers.batch_normalization(C2)
        R2 = lrelu(B2)
        
        C3 = tf.layers.conv2d_transpose(R2, filters = 1, kernel_size = 3,
                                       strides = (1, 1), padding = 'same')
        O = tf.nn.tanh(C3)
        
        return O

In [ ]:
# Structure of the discriminator
def discriminator(X, y_, reuse=None):
    with tf.variable_scope('D',reuse=reuse):
  
        y1 = tf.concat([X, y_], axis = 1)
        
        y2 = tf.layers.dense(y1, units = 256)
        y2 = lrelu(y2)
        y2 = tf.layers.dropout(y2, 0.5)
        
        yout = tf.layers.dense(y2, units = 1)
        O = tf.nn.sigmoid(yout)
        
        return O

In [ ]:
# Structure of regressor
def regressor(X, reuse = None):
    with tf.variable_scope('R', reuse = reuse):
        tower0 = tf.layers.conv2d(X, filters = 32, kernel_size = (1, 1),
                                 padding = 'same')
        
        tower1 = tf.layers.conv2d(X, filters = 64, kernel_size = (1, 1),
                                 padding = 'same')
        tower1 = tf.layers.conv2d(tower1, filters = 64, kernel_size = (3, 3),
                                 padding = 'same')
        
        tower2 = tf.layers.conv2d(X, filters = 32, kernel_size = (1, 1),
                                 padding = 'same')
        tower2 = tf.layers.conv2d(tower2, filters = 32, kernel_size = (5, 5),
                                  padding = 'same')
        
        tower3 = tf.layers.max_pooling2d(X, pool_size = (3, 3), strides = (1, 1),
                                        padding = 'same')
        tower3 = tf.layers.conv2d(tower3, filters = 32, kernel_size = (1, 1),
                                 padding = 'same')
        
        h = tf.concat([tower0, tower1, tower2, tower3], axis = -1)
        h = tf.nn.relu(h)
        h = tf.layers.max_pooling2d(h, pool_size = (2, 2), strides = (1, 1), padding = 'same')
      
        for i in range(6):
            tower0 = tf.layers.conv2d(h, filters = 32, kernel_size = (1, 1),
                                 padding = 'same')
        
            tower1 = tf.layers.conv2d(h, filters = 64, kernel_size = (1, 1),
                                     padding = 'same')
            tower1 = tf.layers.conv2d(tower1, filters = 64, kernel_size = (3, 3),
                                     padding = 'same')

            tower2 = tf.layers.conv2d(h, filters = 32, kernel_size = (1, 1),
                                     padding = 'same')
            tower2 = tf.layers.conv2d(tower2, filters = 32, kernel_size = (5, 5),
                                     padding = 'same')

            tower3 = tf.layers.max_pooling2d(h, pool_size = (3, 3), strides = (1, 1),
                                            padding = 'same')
            tower3 = tf.layers.conv2d(tower3, filters = 32, kernel_size = (1, 1),
                                     padding = 'same')

            h = tf.concat([tower0, tower1, tower2, tower3], axis = -1)
            h = tf.nn.relu(h)
            if i % 2 == 0 and i != 0:
                h = tf.layers.max_pooling2d(h, pool_size = (2, 2), strides = (1, 1), padding = 'same')
        
        Z0 = tf.layers.flatten(h)
        Z0 = tf.layers.dense(Z0, units = 512, activation = 'relu')
        Z0 = tf.layers.dropout(Z0, 0.2)
        
        Z0 = tf.layers.dense(Z0, units = 32)
        Z_out = Z0

        O = tf.layers.dense(Z0, units = 1)
        
        return O, Z_out

In [ ]:
# Helper function of L2 Loss
def L2_Loss(y, y_hat):
    return tf.reduce_mean((y - y_hat) ** 2)

In [ ]:
# Generate data
fake_data = generator(z, y)

# Feed both real and fake data into the regressor for latent features
true_pred, true_logit = regressor(real_data)
fake_pred, fake_logit = regressor(fake_data, reuse = True)

# Use latent feature, bandgap combination to perform authentication
true_label_pred = discriminator(true_logit, y)
fake_label_pred = discriminator(fake_logit, y, reuse = True)

gen_sample = generator(z, y, reuse = True)
gen_pred, gen_logit = regressor(gen_sample, reuse = True)

# Regressor helper loss:
R_loss = L2_Loss(true_pred, y)

# Discriminator losses:
D_real_loss = L2_Loss(true_label_pred, 0.9 * tf.ones_like(true_label_pred))
D_fake_loss = L2_Loss(fake_label_pred, tf.zeros_like(fake_label_pred))
D_loss = (D_real_loss + D_fake_loss)/2

# Generator loss:
G_loss = L2_Loss(fake_label_pred, 0.9 * tf.ones_like(fake_label_pred)) + 25 * L2_Loss(fake_pred, y)

# Opimizer
lr = 1e-4

# Getting variables
tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'D' in var.name]
g_vars = [var for var in tvars if 'G' in var.name]
r_vars = [var for var in tvars if 'R' in var.name]

# Setting up the optimizers
opt_d = tf.train.AdamOptimizer(learning_rate = lr, beta1 = 0.5).minimize(D_loss, var_list = d_vars)
opt_g = tf.train.AdamOptimizer(learning_rate = lr, beta1 = 0.5).minimize(G_loss, var_list = g_vars)
opt_r = tf.train.AdamOptimizer(learning_rate= lr).minimize(R_loss, var_list = r_vars)

In [ ]:
epochs_gan = 256 
batch_size = 60

batches = X_train.shape[0] // batch_size

# Gather the losses for plot
D_Losses = []
G_Losses = []
R_Losses = []

saver = tf.train.Saver()
tol_time = 0

# Training process
with tf.Session(config=session_conf) as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs_gan+1):
        a = time.time()
        D_Losses_ = []
        G_Losses_ = []
        R_Losses_ = []

        for b in range(batches):
            idx = np.arange(b * batch_size, (b + 1) * batch_size)
            train = X_train[idx,:,:,:]
            batch_y = Y_train[idx,:]
            batch_z = np.random.normal(0, 1, size = (batch_size, 128))
            
            # Each round, G, R and D will be trained using a batch of data
            G_loss_, _ = sess.run([G_loss, opt_g], feed_dict = {y: batch_y, z: batch_z})
            R_loss_, _= sess.run([R_loss, opt_r], feed_dict = {real_data: train, y: batch_y})
            D_loss_, _ = sess.run([D_loss, opt_d], feed_dict = {real_data: train, y: batch_y, z: batch_z})
            D_Losses_.append(D_loss_)
            G_Losses_.append(G_loss_)
            R_Losses_.append(R_loss_)
        
        # For timing purposes
        b = time.time()
        delta_t = b - a
        tol_time += delta_t
        
        D_Losses.append(np.mean(D_Losses_))
        G_Losses.append(np.mean(G_Losses_))
        R_Losses.append(np.mean(R_Losses_))
        
        # Generate data and plot loss functions for sanity check every 10 epochs
        if e % 10 == 0:
            print('Currently on epoch {} of {}'.format(e, epochs_gan))
            print('G Loss: {}, D Loss: {}, R Loss: {}'.format(np.mean(G_Losses_), np.mean(D_Losses_), np.mean(R_Losses_)))
            samples = []
            scores = []
            sample_y = np.random.uniform(0.49, 2.15, size = (1, 1))
            sample_y = np.round(sample_y, 4)
            sample_y_ = sample_y * np.ones([100, 1])
            sample_z = np.random.normal(0, 1, size = (100, 128))
                
            gen_sample_, pred_score = sess.run([gen_sample, gen_pred], feed_dict = {y: sample_y_, z: sample_z})

            samples = np.array(gen_sample_)
            samples = samples.reshape([100,4,4,1])
            samples = np.where(samples < 0, -1, 1)
            
            S_ = samples.tolist()
            not_in = 0
            for s_ in S_:
                if s_ not in X_train.tolist():
                    not_in += 1
            
            print('Current number of unique samples: {}'.format(np.shape(np.unique(samples[:,:,:,0], axis = 0))[0]))
            print('------------')
            print('Currently Generating {}'.format(sample_y))
            print('Predicted bandgap: {}'.format(np.mean(pred_score)))
            print('Samples not in database :{}'.format(not_in))
            print('------------')
            
            if e != 0:
                plt.plot(G_Losses)
                plt.plot(D_Losses)
                plt.plot(R_Losses)
                plt.legend(['G Loss', 'D Loss', 'R Loss'])
                plt.show()
        
        # Save the model every 10 epochs
        if e % 10 == 0:
            print('-----------')
            print('Saving current model')
            saver.save(sess, "cGAN_model.ckpt")
            with open('cGAN_Losses.pickle', 'wb') as f:
                pickle.dump((G_Losses, D_Losses, R_Losses), f)

In [ ]:
# Average time spent on training
print('average running time: {}'.format(tol_time / 256))

In [ ]:
# Check on the accuracy of the regressor
saver = tf.train.Saver()
batch_size = 60

batches = X_train.shape[0] // batch_size
preds = []

with tf.Session(config = session_conf) as sess:
    saver.restore(sess, 'cGAN_model.ckpt')
    for b in range(batches):
        if b % 50 == 0:
            print('Currently: {}/{}'.format(b, batches))
        idx = np.arange(b * batch_size, (b + 1) * batch_size)
        data = X_train[idx,:,:,:]
        predicted, _ = sess.run(regressor(real_data, reuse = True), feed_dict = {real_data: data})
        
        preds.extend(predicted.reshape([-1]))

In [ ]:
# Fractional MAE for regressor
preds = np.asarray(preds)
print('Fractional MAE: {}'.format(np.mean(np.abs((preds - Y_train.reshape([-1])) / Y_train.reshape([-1])))))

In [ ]:
# Generate new data for testing purposes

OUT = pd.DataFrame()

saver = tf.train.Saver()

for K in range(5 + 1):
    print('Progress: {}/{}'.format(K, 5))
    with tf.Session(config=session_conf) as sess:
        saver.restore(sess, 'cGAN_model.ckpt')
        print('------------')
        Samples = []
        labels = []
        Inception_scores = []
        gen_preds = []

        for i in range(50):
            labels_pred = []
            sample_y = np.random.uniform(0.49, 2.15, size = (1, 1))
            sample_y = np.round(sample_y, 4)
            sample_y_ = sample_y * np.ones([100, 1])

            sample_z = np.random.normal(0, 1, size = (100, 128))
            gen_sample = sess.run(generator(z, y, reuse = True), feed_dict = {z: sample_z, y: sample_y_})

            gen_sample = np.where(gen_sample < 0, -1, 1)
            gen_score = sess.run(R_loss, feed_dict = {real_data: gen_sample, y: sample_y_})

            Inception_scores.extend(list(np.unique(gen_score)))

            gen_pred_,_ = sess.run(regressor(real_data, reuse = True), feed_dict = {real_data: np.unique(gen_sample, axis = 0)})
            gen_preds.append(gen_pred_)

            # print('------------')
            gen_sample = np.array(gen_sample)
            gen_sample = gen_sample.reshape([100,4,4,1])
            Samples.extend(list(np.unique(gen_sample, axis = 0)))
            labels.extend(list(sample_y * np.ones(shape = (1, np.shape(np.unique(gen_sample[:,:,:,0], axis = 0))[0]))))

    labels_ = []
    for l in labels:
        for j in range(len(l)):
            labels_.append(l[j])
            
    Gen_preds_ = []
    for l in gen_preds:
        for j in range(len(l)):
            Gen_preds_.append(l[j])
            
    Samples_ = np.where(np.array(Samples)<0, 0, 1)
    
    summary = {}
    summary['X'] = []
    summary['require_label'] = []
    summary['predicted_label'] = []

    for s, l, gen_p in zip(Samples_, labels_, Gen_preds_):
        summary['X'].append(int('0b'+''.join(list(s.reshape([16,]).astype('str').tolist())), 2))
        summary['require_label'].append(l)
        summary['predicted_label'].append(gen_p[0])
        
    out = pd.DataFrame(summary)
    out = out.sort_values('X').reset_index().drop('index', axis = 1)

    OUT = pd.concat([OUT, out], axis = 0, ignore_index = True)
    
    with open('summary.pickle', 'wb') as f:
        pickle.dump(OUT, f)

In [ ]:
# Fractional MAE for generation
r = OUT.require_label.values
p = OUT.predicted_label.values

print('Fractional MAE: {}'.format(np.mean(np.abs((p - r) / r))))

In [ ]:
OUT.to_csv('summary.csv', index = False)